In [1]:
from main import main
from utils import LoadData, LoadModel, load_data
import argparse
import numpy as np
import time
import copy
import os
import tempfile

import pandas as pd
# pyomo for optimization
import pyomo.environ as pyo

# pytorch for training neural network
import torch.onnx
from torch import nn, optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

# omlt for interfacing our neural network with pyomo
import onnx
from omlt import OffsetScaling, OmltBlock
from omlt.io.onnx import (
    load_onnx_neural_network_with_bounds,
    write_onnx_model_with_bounds,
    load_onnx_neural_network,
)
from omlt.neuralnet import (FullSpaceNNFormulation, 
    ReluComplementarityFormulation, 
    ReluPartitionFormulation,
    ReducedSpaceNNFormulation)

2025-01-11 20:51:00.580860: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-11 20:51:00.593460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736650260.606070  965702 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736650260.609373  965702 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-11 20:51:00.623484: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def add_arguments(model,model_id,system,scenario):    
    if system == 'membrane':
        args = argparse.Namespace(
            input_dim=7,
            hidden_dim=32,
            hidden_num=2,
            z0_dim=8,
            optimizer='adam',
            epochs=1000,
            batch_size=16,
            lr=1e-4,
            mu=1,
            max_subiter=500,
            eta=0.8,
            sigma=2,
            mu_safe=1e+9,
            dtype=32,
            dataset_path='/home/andresfel9403/KKThNN/KKThPINN/benchmark_membrane.csv',
            val_ratio=0.2,
            job='train',
            runs=10)
    else:
        pass

    args.model = model
    args.model_id = model_id
    args.dataset_type = system
    args.scenario = scenario
        
    
    if args.model == 'NN':
        args.loss_type = 'MSE'
    elif args.model == 'PINN':
        args.loss_type = 'PINN'
    elif args.model == 'KKThPINN':
        args.loss_type = 'MSE'
    elif args.model == 'AugLagNN':
        args.loss_type = 'MSE'
    elif args.model == 'ECNN':
        args.loss_type = 'MSE'
    return args

In [4]:
inputs = ['F_MeOH',
    'F_DME',
    'F_H2O',
    'F_N2',
    'T_in',
    'P_in',
    'T_permeate']

outputs = [
    'F_MeOH_O',
    'F_DME_O',
    'F_H2O_O',
    'F_H2O_M_O',
    'F_N2_O',
    'T_Out',
    'T_Out_M',
    'P_Out'
]

columns = inputs+outputs

df = pd.read_csv("benchmark_membrane.csv", usecols = columns)


In [5]:
args = add_arguments('KKThPINN','Membrane_KKT_NN','membrane','demonstration')

In [6]:
print(args.model == 'KKThPINN')

True


In [7]:
main(args)

type of A: torch.float32, type of B: torch.float32, type of b: torch.float32
train set size: 614, val set size: 204, test set size: 204
Start Training...
epoch: 00050 loss_train: 0.00642 loss_val: 0.00668 violation_train: 0.00000 violation_val: 0.00000
epoch: 00100 loss_train: 0.00109 loss_val: 0.00153 violation_train: 0.00000 violation_val: 0.00000
epoch: 00150 loss_train: 0.00069 loss_val: 0.00110 violation_train: 0.00000 violation_val: 0.00000
epoch: 00200 loss_train: 0.00059 loss_val: 0.00095 violation_train: 0.00000 violation_val: 0.00000
epoch: 00250 loss_train: 0.00053 loss_val: 0.00088 violation_train: 0.00000 violation_val: 0.00000


KeyboardInterrupt: 

After defining the arguments depending on the number of inputs, hidden and output layers, we can train the model selecting the Neural Network model (NN or KKThPINN)

In [8]:
dataset, scaling = load_data(args.dataset_path)
inputs = dataset[:,:7]
outputs = dataset[:,7:]

data = LoadData(args)
model_NN = LoadModel(args, data)
PATH = '/home/andresfel9403/KKThNN/models/membrane/KKThPINN/0.2/None_0.2_9.pth'
checkpoint = torch.load(PATH)
model_NN.load_state_dict(checkpoint['state_dict'])
#model_NN.eval()



type of A: torch.float32, type of B: torch.float32, type of b: torch.float32
train set size: 614, val set size: 204, test set size: 204


/tmp/ipykernel_965702/2836315970.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)


<All keys matched successfully>

In [9]:
x_factor = scaling.scale_[:7]
y_factor = scaling.scale_[7:]

print(x_factor)

[1.46831994e+01 1.04111445e+01 1.42231966e+01 9.99584502e+01
 7.26638589e+02 2.69926289e+06 1.76773269e+02]


In [10]:
scaler = scaler = OffsetScaling(
    offset_inputs={i: 0 for i in range(len(x_factor))},
    factor_inputs={i: x_factor[i] for i in range(len(x_factor))},
    offset_outputs={i: 0 for i in range(len(y_factor))},
    factor_outputs={i: y_factor[i] for i in range(len(y_factor))},
)

In [11]:
x_dummy = torch.from_numpy(inputs[0]).float()
ub = np.max(inputs, 0)
lb = np.min(inputs, 0)

scaled_input_bounds = {i: (lb[i], ub[i]) for i in range(len(inputs[0]))}
print(ub.dtype)

float64


In [12]:
def transfer_weights(original_model, modified_model):
    with torch.no_grad():
        for original_layer, modified_layer in zip(original_model.layers, modified_model.layers):
            if isinstance(original_layer, nn.Linear) and isinstance(modified_layer, nn.Linear):
                modified_layer.weight.copy_(original_layer.weight)
                modified_layer.bias.copy_(original_layer.bias)

In [13]:
def _create_onnx_model(data,args,model,file_path,x,input_bounds):
    args.model='NN'
    print('Saving standard model')
    modified_model = LoadModel(args,data)
    transfer_weights(model,modified_model)
    create_onnx_model(data,modified_model,file_path,x,input_bounds)

In [14]:
def create_onnx_model(data,model,file_path,x,input_bounds):
    
    torch.onnx.export(
        model,
        x,
        file_path,
        input_names=["input"],
        dynamo = False
    )
    write_onnx_model_with_bounds(file_path, None, input_bounds)
    print(f"Wrote PyTorch Onnx model to {file_path}")

In [ ]:
_create_onnx_model(data,args,model_NN,'dist33.onnx',x_dummy,scaled_input_bounds)

Saving standard model


In [28]:
#with tempfile.NamedTemporaryFile(suffix=".onnx", delete=False) as f:
    # export neural network to ONNX
torch.onnx.export(
    model_NN,
    x_dummy,
    'intento.onnx',
    input_names=["input"],
    dynamo = False
)
# write ONNX model and its bounds using OMLT
write_onnx_model_with_bounds('intento.onnx', None, scaled_input_bounds)
onnx_model = onnx.load('intento.onnx')
# load the network definition from the ONNX model
#network_definition = load_onnx_neural_network_with_bounds('intento.onnx')
network_definition = load_onnx_neural_network(onnx = onnx_model,  scaling_object=scaler, input_bounds= scaled_input_bounds) 

In [17]:
network_definition = load_onnx_neural_network_with_bounds('dist33.onnx') 


ValueError: Unhandled node type Add

In [108]:
formulation = FullSpaceNNFormulation(network_definition)

In [ ]:
for layer_id, layer in enumerate(network_definition.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

In [ ]:
model = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
model.membrane = OmltBlock()
model.membrane.build_formulation(formulation)

# model.sections = pyo.Set(initialize=["retentate", "permeate"], doc="sections in the model")
# model.components = pyo.Set(initialize=["MeOH", "DME", "H2O", "N2"], doc="Components in the model")


In [ ]:
model.bounds = Param (
    model.bounds_set,
    initialize = {
        'Tlow': 460, 'Thi': 1000,
        'Tmlow': 283.15, 'Tmhi': 450,
        'Ftmlow': 1, 'Ftmhi': 100,
        'Plow': 13e5, 'Phi': 27e5,
    },
    doc = "Bounds for the variables"
)

In [110]:
model.obj = pyo.Objective(expr=model.membrane.outputs[1], sense=pyo.maximize)

In [ ]:
solver = pyo.SolverFactory("cbc")
status = solver.solve(model, tee=True)

In [ ]:
print("FA:", pyo.value(model.membrane.inputs[4]))

In [ ]:
print(x_factor)